In [7]:
# ------------------------------------------------------------
# Cut to Ship Report Automation
# Robust to column name variations
# ------------------------------------------------------------

import pandas as pd
import numpy as np
import re

# -----------------------------
# Helper functions for cleaning and standardizing inputs
# -----------------------------
def clean_cols(df: pd.DataFrame) -> pd.DataFrame:
    """Normalizing column names by removing extra whitespace and trimming edges."""
    df = df.copy()
    df.columns = (
        df.columns.astype(str)
        .str.replace(r"\s+", " ", regex=True)  # collapsing whitespace/newlines/tabs
        .str.strip()
    )
    return df

def find_col(df: pd.DataFrame, text: str, prefer_exact: bool = True) -> str:
    """
    Identifying the correct column even if the column name slightly varies.
    - Trying exact match first (case-insensitive)
    - Otherwise selecting the first column that contains the keyword
    """
    cols = list(df.columns)

    if prefer_exact:
        for c in cols:
            if c.strip().lower() == text.strip().lower():
                return c

    matches = [c for c in cols if text.lower() in c.lower()]
    if not matches:
        raise KeyError(
            f"Could not find a column containing '{text}'. Available columns:\n{cols}"
        )
    return matches[0]

def ensure_datetime(s: pd.Series) -> pd.Series:
    """Converting values to datetime format safely (invalid values become NaT)."""
    return pd.to_datetime(s, errors="coerce")

def safe_str(s: pd.Series) -> pd.Series:
    """Converting to string while keeping missing values safe for concatenations."""
    return s.fillna("").astype(str)

# -----------------------------
# Defining file paths for the 7 source reports
# -----------------------------
D1_path = r"D:\Savidhu_OneDrive\OneDrive - Hirdaramani Group\Projects\Cut to Ship Prediction Model\Cut to Ship Report Automation\Style Closure -Week 43.xlsx"
D2_path = r"D:\Savidhu_OneDrive\OneDrive - Hirdaramani Group\Projects\Cut to Ship Prediction Model\Cut to Ship Report Automation\Garment Sales Order-Week 43.xlsx"
D3_path = r"D:\Savidhu_OneDrive\OneDrive - Hirdaramani Group\Projects\Cut to Ship Prediction Model\Cut to Ship Report Automation\Order Book -Week 43.xlsx"
D4_path = r"D:\Savidhu_OneDrive\OneDrive - Hirdaramani Group\Projects\Cut to Ship Prediction Model\Cut to Ship Report Automation\Transaction Summary -Week 43.xlsx"
D5_path = r"D:\Savidhu_OneDrive\OneDrive - Hirdaramani Group\Projects\Cut to Ship Prediction Model\Cut to Ship Report Automation\Export Summary -Week 43.xlsx"
D6_path = r"D:\Savidhu_OneDrive\OneDrive - Hirdaramani Group\Projects\Cut to Ship Prediction Model\Cut to Ship Report Automation\Last Shipment- Week 43.xlsx"
D7_path = r"D:\Savidhu_OneDrive\OneDrive - Hirdaramani Group\Projects\Cut to Ship Prediction Model\Cut to Ship Report Automation\Master sheet - customer name.xlsx"

# -----------------------------
# Loading all source datasets into pandas DataFrames
# -----------------------------
D1 = pd.read_excel(D1_path, sheet_name="Sheet1")
D2 = pd.read_excel(D2_path, sheet_name="Sheet1")
D3 = pd.read_excel(D3_path, sheet_name="Sheet1")
D4 = pd.read_excel(D4_path, sheet_name="Sheet1")
D5 = pd.read_excel(D5_path, sheet_name="Sheet1")
D6 = pd.read_excel(D6_path, sheet_name="Sheet1")
D7 = pd.read_excel(D7_path, sheet_name="Sheet1")

# Cleaning column headers to make matching reliable
D1 = clean_cols(D1)
D2 = clean_cols(D2)
D3 = clean_cols(D3)
D4 = clean_cols(D4)
D5 = clean_cols(D5)
D6 = clean_cols(D6)
D7 = clean_cols(D7)

# -----------------------------
# Resolving the required column names dynamically (robust to naming variations)
# -----------------------------
# Resolving D2 filter columns
D2_gen = find_col(D2, "General sales order")
D2_sample = find_col(D2, "Sample sales orders")
D2_salesman = find_col(D2, "Salesman order")
D2_sales_order = find_col(D2, "Sales order")  # could be Sales order, Sales order.1, etc.

# Resolving D1 columns
D1_sales_order = find_col(D1, "Sales order")
D1_customer = find_col(D1, "Customer account")
D1_style_closed = find_col(D1, "Style closed date")

# Resolving D7 columns
D7_customer = find_col(D7, "Customer")
D7_calling = find_col(D7, "Calling Name")

# Resolving D3 columns
D3_sales_order = find_col(D3, "Sales order")
D3_division = find_col(D3, "Division")
D3_season = find_col(D3, "Season")
D3_style_no = find_col(D3, "Style number")
D3_item_type = find_col(D3, "Garment item type")
D3_site = find_col(D3, "Site")
D3_set_garment = find_col(D3, "Set garment")
D3_qty = find_col(D3, "Quantity")

# Resolving D4 columns
D4_sales_order = find_col(D4, "Sales order")
D4_unit = find_col(D4, "Unit")
D4_qty = find_col(D4, "Qty")

# Resolving D5 columns
D5_sales_order = find_col(D5, "Sales order")
D5_site = find_col(D5, "Site")
D5_invoice = find_col(D5, "Customer invoice")
D5_date = find_col(D5, "Date")
D5_invoice_qty = find_col(D5, "Invoice qty")
D5_fob = find_col(D5, "FOB")

# Resolving D6 columns
D6_sales_order = find_col(D6, "Sales order")
D6_approved_date = find_col(D6, "Approved date")

# Converting date fields into datetime format
D1[D1_style_closed] = ensure_datetime(D1[D1_style_closed])
D6[D6_approved_date] = ensure_datetime(D6[D6_approved_date])
D5[D5_date] = ensure_datetime(D5[D5_date])

# -----------------------------
# Filtering bulk sales orders from D2 (keeping only rows where all 3 flags are "No")
# -----------------------------
filtered_D2 = D2[
    (D2[D2_gen] == "No") &
    (D2[D2_sample] == "No") &
    (D2[D2_salesman] == "No")
][[D2_sales_order]].copy()

# -----------------------------
# Filtering D1 to keep only matching bulk sales orders (semi-join logic)
# -----------------------------
filtered_D1 = D1[D1[D1_sales_order].isin(filtered_D2[D2_sales_order])].copy()

# -----------------------------
# Initializing the result dataset using filtered Sales Orders
# -----------------------------
result = filtered_D1[[D1_sales_order]].copy()
result = result.rename(columns={D1_sales_order: "Sales_order"})

# -----------------------------
# Adding Customer details from D1
# -----------------------------
result = result.merge(
    D1[[D1_sales_order, D1_customer]].rename(columns={D1_sales_order: "Sales_order", D1_customer: "Customer"}),
    on="Sales_order",
    how="left"
)

# -----------------------------
# Adding Calling Name from master mapping (D7) using Customer
# -----------------------------
result = result.merge(
    D7[[D7_customer, D7_calling]].rename(columns={D7_customer: "Customer", D7_calling: "Calling_Name"}),
    on="Customer",
    how="left"
)

# -----------------------------
# Adding order attributes from Order Book (D3) using Sales Order
# -----------------------------
result = result.merge(
    D3[[D3_sales_order, D3_division, D3_season, D3_style_no, D3_item_type, D3_site, D3_set_garment]].rename(
        columns={
            D3_sales_order: "Sales_order",
            D3_division: "Division",
            D3_season: "Season",
            D3_style_no: "Style_number",
            D3_item_type: "Garment_item_type",
            D3_site: "Unit",
            D3_set_garment: "Set_garment",
        }
    ),
    on="Sales_order",
    how="left"
)

# -----------------------------
# Adding "Last Shipped" date from D6 by taking the latest Approved date per Sales order
# -----------------------------
last_shipped = (
    D6.groupby(D6_sales_order, as_index=False)[D6_approved_date]
    .max()
    .rename(columns={D6_sales_order: "Sales_order", D6_approved_date: "Last_Shipped"})
)

result = result.merge(last_shipped, on="Sales_order", how="left")

# -----------------------------
# Adding "Style closed date" from D1
# -----------------------------
result = result.merge(
    D1[[D1_sales_order, D1_style_closed]].rename(columns={D1_sales_order: "Sales_order", D1_style_closed: "Style_closed_date"}),
    on="Sales_order",
    how="left"
)

# -----------------------------
# Deriving Month and Week from Style closed date
# -----------------------------
result["Month"] = result["Style_closed_date"].dt.month_name().str[:3]

iso_week = result["Style_closed_date"].dt.isocalendar().week.astype("Int64")
result["Week"] = iso_week.apply(lambda x: f"Week {int(x):02d}" if pd.notna(x) else np.nan)

# -----------------------------
# Deriving Operation type based on Sales order prefix
# -----------------------------
so_str = safe_str(result["Sales_order"])
result["Operation"] = np.where(
    so_str.str.startswith("N"), "Knit Operation",
    np.where(so_str.str.startswith("W"), "Woven Operation", "Other Operation")
)

# -----------------------------
# Removing excluded Customers from the dataset
# -----------------------------
result = result[~result["Customer"].astype(str).str.lower().str.contains("oritapparels|southasiatextiles", regex=True, na=False)].copy()

# -----------------------------
# Creating a unique "Code" key by combining Sales order and Unit
# -----------------------------
result["Code"] = safe_str(result["Sales_order"]) + safe_str(result["Unit"])

D3["Code"] = safe_str(D3[D3_sales_order]) + safe_str(D3[D3_site])
D4["Code"] = (safe_str(D4[D4_sales_order]) + safe_str(D4[D4_unit])).str.upper()
D5["Code"] = safe_str(D5[D5_sales_order]) + safe_str(D5[D5_site])

# -----------------------------
# Aggregating Order Qty, Cut Qty, and Ship Qty by Code
# -----------------------------
order_qty = (
    D3[~D3[D3_site].astype(str).str.contains("HIKH-SAMP|HKSAM", regex=True, na=False)]
    .groupby("Code", as_index=False)[D3_qty]
    .sum()
    .rename(columns={D3_qty: "Order_Qty"})
)

cut_qty = (
    D4.groupby("Code", as_index=False)[D4_qty]
    .sum()
    .rename(columns={D4_qty: "Cut_Qty"})
)

ship_qty = (
    D5[
        ~D5[D5_invoice].astype(str).str.lower().str.match(r"^(scl|rtn|dummy|sms|ss)", na=False)
    ]
    .drop_duplicates(subset=[D5_invoice, D5_date, D5_invoice_qty])
    .groupby("Code", as_index=False)[D5_invoice_qty]
    .sum()
    .rename(columns={D5_invoice_qty: "Ship_Qty"})
)

# Merging aggregated quantities into the result dataset
result = result.merge(order_qty, on="Code", how="left")
result = result.merge(cut_qty, on="Code", how="left")
result = result.merge(ship_qty, on="Code", how="left")

# -----------------------------
# Assigning Sales Order type
# -----------------------------
result["SO_Type"] = "Bulk"

# -----------------------------
# Deriving Pcs and adjusting Cut Qty for pack styles
# -----------------------------
set_g = result["Set_garment"].astype(str)
style_num = result["Style_number"].astype(str)

pcs = np.where(
    set_g.eq("Single"), 1,
    np.where(
        set_g.str.contains("Pack", na=False),
        pd.to_numeric(style_num.str.extract(r"(\d+)(?=PK|P)")[0], errors="coerce"),
        np.nan
    )
)

result["Pcs"] = pcs.astype(float)
result.loc[result["Pcs"] > 10, "Pcs"] = np.nan

result["Cut_Qty"] = pd.to_numeric(result["Cut_Qty"], errors="coerce")
result.loc[result["Pcs"].notna(), "Cut_Qty"] = result.loc[result["Pcs"].notna(), "Cut_Qty"] / result.loc[result["Pcs"].notna(), "Pcs"]

# -----------------------------
# Removing duplicate Codes (keeping the first record per Code)
# -----------------------------
result = result.drop_duplicates(subset=["Code"], keep="first").copy()

# -----------------------------
# Calculating Cut/Ship, Order/Ship, and Order/Cut ratios
# -----------------------------
result["Cut/Ship"] = result["Ship_Qty"] / result["Cut_Qty"]
result["Order/Ship"] = result["Ship_Qty"] / result["Order_Qty"]
result["Order/Cut"] = result["Cut_Qty"] / result["Order_Qty"]

# -----------------------------
# Calculating FOB (sum of FOB * Invoice qty) by Code
# -----------------------------
D5_fob_total = pd.to_numeric(D5[D5_fob], errors="coerce") * pd.to_numeric(D5[D5_invoice_qty], errors="coerce")
fob_data = (
    D5.assign(FOB_Total=D5_fob_total)
    .groupby("Code", as_index=False)["FOB_Total"]
    .sum()
    .rename(columns={"FOB_Total": "FOB"})
)

result = result.merge(fob_data, on="Code", how="left")

# -----------------------------
# Preparing the final output structure and column naming
# -----------------------------
result_out = result.rename(columns={
    "Sales_order": "Sales order",
    "Calling_Name": "Calling Name",
    "Division": "Div",
    "Style_number": "Style number",
    "Garment_item_type": "Garment item type",
    "Last_Shipped": "Last Shipped",
    "Style_closed_date": "Style closed date",
    "Order_Qty": "Order Qty",
    "Cut_Qty": "Cut Qty",
    "Ship_Qty": "Ship Qty",
    "SO_Type": "SO Type",
    "Set_garment": "Set garment",
})

# -----------------------------
# Filtering output to only include Style closed date in 2024
# -----------------------------
result_out["Style closed date"] = ensure_datetime(result_out["Style closed date"])
result_out = result_out[result_out["Style closed date"].dt.year == 2024].copy()

# -----------------------------
# Filling missing Cut Qty values using Ship Qty where applicable
# -----------------------------
result_out["Cut Qty"] = pd.to_numeric(result_out["Cut Qty"], errors="coerce")
result_out["Ship Qty"] = pd.to_numeric(result_out["Ship Qty"], errors="coerce")
result_out.loc[result_out["Cut Qty"].isna() & result_out["Ship Qty"].notna(), "Cut Qty"] = result_out.loc[
    result_out["Cut Qty"].isna() & result_out["Ship Qty"].notna(), "Ship Qty"
]

# -----------------------------
# Excluding Corporate and Sample units from the final output
# -----------------------------
result_out["Unit"] = result_out["Unit"].astype(str)
result_out = result_out[~result_out["Unit"].str.contains("Corporate|Sample", regex=True, na=False)].copy()

# -----------------------------
# Selecting and ordering final columns for export
# -----------------------------
final_cols = [
    "Sales order",
    "Customer",
    "Calling Name",
    "Div",
    "Season",
    "Style number",
    "Garment item type",
    "Unit",
    "Last Shipped",
    "Style closed date",
    "Month",
    "Order Qty",
    "Cut Qty",
    "Ship Qty",
    "Cut/Ship",
    "Order/Ship",
    "Order/Cut",
    "SO Type",
    "Week",
    "Operation",
    "Set garment",
    "Pcs",
    "Code",
    "FOB",
]

final_cols_existing = [c for c in final_cols if c in result_out.columns]
result_out = result_out[final_cols_existing].copy()

# -----------------------------
# Writing the final Cut to Ship report output to Excel
# -----------------------------
output_path = (
    r"D:\Savidhu_OneDrive\OneDrive - Hirdaramani Group\Projects"
    r"\Cut to Ship Prediction Model\Cut to Ship Report Automation"
    r"\Cut_to_Ship_Week_43_Test.xlsx"
)

result_out.to_excel(output_path, index=False)
print("Saved:", output_path)


Saved: D:\Savidhu_OneDrive\OneDrive - Hirdaramani Group\Projects\Cut to Ship Prediction Model\Cut to Ship Report Automation\Cut_to_Ship_Week_43_Test.xlsx
